In [1]:
from quantum_utils import (
    H7,H5,
    poly_log_t,
    H_stab_state_evol,
    stab_01n_op,
    ini_state_01,
    plot_x_y,
    train_linear_svm_pauli,
    classify_state_with_svm,
    sample_random_pure_separable_state
)
from qiskit.quantum_info import SparsePauliOp,Statevector
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
n = 7
H = H7
stab_list = stab_01n_op
ini_state = ini_state_01
steps = 10
t = poly_log_t
limit = 0.01
turns = 1
max_round = 50
num_s = 10000
class_w = (1,1) #(separable, entangled)
mar_neg = 1.0
mar_pos = 1.0

In [3]:
test_ob = H_stab_state_evol(n, H7, stab_01n_op, ini_state_01, poly_log_t, steps)

test_ob.state_evol(ideal_state=True, den_mat_out=False, state_update=True)

test_ob.stablt_sum_evol(approx=True, up=limit, step_by_step=True, stab_sum_update=True)

SparsePauliOp(['IIIIIIZ', 'IIIIIXY', 'IIIIIYX', 'XIIIIXZ', 'IIIIIZI', 'ZIIIIII', 'YIIIIYZ', 'IIIIZII', 'IIIZIII', 'IIZIIII', 'IZIIIII', 'IIIIYZY', 'IIIIYXI', 'IIIIXZX', 'IIIIXYI', 'IIIXYII', 'IIIYXII', 'IIXYIII', 'IIYXIII', 'IXYIIII', 'IYXIIII', 'ZXIIIIX', 'YXIIIII', 'XYIIIII', 'ZYIIIIY', 'IIIXZXI', 'YZYIIII', 'IIIYZYI', 'XZXIIII', 'IIXZXII', 'IIYZYII', 'IXZXIII', 'IYZYIII'],
              coeffs=[ 0.95862348-1.64999951e-10j, -0.28092041-2.14436367e-09j,
  0.28092041+2.14436367e-09j, -0.02057946-1.25103105e-10j,
 -0.91746438+4.75026382e-10j,  0.95862348-1.64999951e-10j,
 -0.02057946-1.25103105e-10j,  0.91768206-4.68392576e-10j,
 -0.91768155+4.77743688e-10j,  0.91768206-4.68392576e-10j,
 -0.91746438+4.75026382e-10j,  0.04101355+1.08485998e-09j,
 -0.2779266 -1.85700685e-09j,  0.04101355+1.08485998e-09j,
  0.2779266 +1.85700685e-09j, -0.27793716-1.98456856e-09j,
  0.27793716+1.98456856e-09j,  0.27793716+1.98456856e-09j,
 -0.27793716-1.98456856e-09j, -0.2779266 -1.85700685e-09j,
  0.277926

In [5]:
test_ob.SVM_linear(num_s, class_w)

In [4]:
test_ob.SVM_linear(num_samples = num_s, class_weights=class_w, mar_neg= mar_neg, mar_pos= mar_pos)
label, result = test_ob.test_EW_SVM()

tested result: ('separable', -0.9915960627987889)


In [11]:
phi_appro = (test_ob.stabsum_tog + SparsePauliOp.from_list([('I' * test_ob.n, 1 + 0.j)])) * (2 ** (- test_ob.n))
model = train_linear_svm_pauli(7, phi_appro, 10000)


KeyboardInterrupt: 

In [6]:
model = test_ob.svm_linear_model

In [7]:
test = Statevector.to_operator(test_ob.ini_state).to_matrix()
sample = sample_random_pure_separable_state(7)

In [8]:
a = classify_state_with_svm(test, model, return_score = True)
b = classify_state_with_svm(sample, model, return_score = True)

In [9]:
c = classify_state_with_svm(2**(-7)*test_ob.stabsum_tog.to_matrix(), model, return_score = True)
d = classify_state_with_svm(2**(-7)* (test_ob.stabsum_tog.to_matrix() + SparsePauliOp.from_list([('I' * 7, 1 +0.j)]).to_matrix()), model, return_score = True)

In [ ]:
print(np.linalg.norm(test))

In [10]:
print(np.linalg.norm(test - 2**(-7)*test_ob.stabsum_tog.to_matrix(), ord=2))
print(np.linalg.norm(test - 2**(-7)* (test_ob.stabsum_tog.to_matrix() + SparsePauliOp.from_list([('I' * 7, 1 +0.j)]).to_matrix()), ord=2))

0.9453135514479676
0.9375010514479677


In [10]:
print(a,b,c,d)

('separable', -0.9905037542478133) ('separable', -1.008166130181369) ('separable', -0.9905036867828909) ('separable', -0.9905036867828909)


In [ ]:
('separable', -0.9900779150887992) ('separable', -1.0034439287265375) ('entangled', 0.9999999474531118) ('entangled', 0.9999999474531203)
